<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/custom_entity_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
!pip install spacymoji
from spacymoji import Emoji

     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=edebb9c9614958a390b2d6ba2f7b10254c2e5219601b4654d79d39ac35b636e4
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
import random
import srsly
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.util import minibatch, compounding
from pandas.io.json import json_normalize

In [5]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
from spacy.gold import docs_to_json, biluo_tags_from_offsets, offsets_from_biluo_tags

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
def cleanse_text(text):
  step1 = re.sub(r"(\{'LOWER':\s)",r"",str(text))
  step2 = re.sub(r"[\[{\'\,\"}\]]",r"",step1)
  return step2

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']

        print(text, annotations)
        valid_entities = []
        for start, end, label in entities:
          valid_start = start
          valid_end = end
          print(valid_start, valid_end, len(text))
          while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
            valid_start += 1
          while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
            valid_end -= 1
          valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [8]:
df_pattern = pd.read_json('patterns.jsonl',lines=True)
df_pattern['cleaned_pattern'] = df_pattern.pattern.apply(lambda x: cleanse_text(x))
df_pattern['label'] = 'U_' + df_pattern.label.astype(str)
df_pattern['tup_col'] = list(zip(df_pattern.cleaned_pattern, df_pattern.label))
df_pattern['merged'] = df_pattern.apply(lambda row: {row['cleaned_pattern']:row['label']}, axis=1)
df_pattern.head()

,label,pattern,id,cleaned_pattern,tup_col,merged
0,U_dish,[{'LOWER': ''mpanatigghi'}],'mpanatigghi,mpanatigghi,"(mpanatigghi, U_dish)",{'mpanatigghi': 'U_dish'}
1,U_dish,[{'LOWER': ''nduja'}],'nduja,nduja,"(nduja, U_dish)",{'nduja': 'U_dish'}
2,U_dish,"[{'LOWER': ''ota'}, {'LOWER': ''ika'}]",'ota_'ika,ota ika,"(ota ika, U_dish)",{'ota ika': 'U_dish'}
3,U_dish,"[{'LOWER': '.amaro'}, {'LOWER': 'ramazzotti'}]",.amaro_ramazzotti,.amaro ramazzotti,"(.amaro ramazzotti, U_dish)",{'.amaro ramazzotti': 'U_dish'}
4,U_dish,"[{'LOWER': '15'}, {'LOWER': 'bean'}, {'LOWER':...",15_bean_soup,15 bean soup,"(15 bean soup, U_dish)",{'15 bean soup': 'U_dish'}


In [9]:
ent_list = df_pattern.tup_col.to_list()
ent_list[50:60]

[('agliata', 'U_dish'),
 ('agneau du périgord', 'U_dish'),
 ('agnolotti', 'U_dish'),
 ('agrodolce', 'U_dish'),
 ('agua de sevilla', 'U_dish'),
 ('agua de valencia', 'U_dish'),
 ('aguachile', 'U_dish'),
 ('aguadito de pollo', 'U_dish'),
 ('aguapanela', 'U_dish'),
 ('aguas frescas', 'U_dish')]

In [0]:
nlp = spacy.load("en_core_web_lg")
ruler = EntityRuler(nlp).from_disk("patterns.jsonl")
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)
nlp.add_pipe(ruler, before='ner')
merge_ents = nlp.create_pipe("merge_entities")
nlp.add_pipe(merge_ents, after='ner')

In [11]:
nlp.pipe_names

['emoji', 'tagger', 'parser', 'entity_ruler', 'ner', 'merge_entities']

In [12]:
doc1 = nlp("I love fettuccine cheese 💚🌿 but hate buffalo wings.")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc1.ents])

displacy.render(doc1,'ent',jupyter=True)



[('fettuccine', 'dish', 'fettuccine'), ('buffalo wings', 'dish', 'buffalo_wings')]


In [13]:
doc2 = nlp("Avocado 🌿 are good but chicken tikka masala is fantastic while paneer tikka masala and sweet potato fries are atrocious")
displacy.render(doc2,'ent',jupyter=True)

In [0]:
nlp.pipe_names

['emoji', 'tagger', 'parser', 'entity_ruler', 'ner']

In [14]:
doc3 = nlp("Baguette is the best bread in the whole world - bagels come close and ciabatta is also up there")
displacy.render(doc3,'ent',jupyter=True)

In [0]:
nlp.disable_pipes('entity_ruler')

[('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x7f5fdaccc630>)]

In [0]:
doc3 = nlp("Baguette is the best chaunk bread in the whole world - bagels come close and ciabatta is also up there")
displacy.render(doc3,'ent',jupyter=True)

In [15]:
reviews = pd.read_csv('reviews_text.csv',usecols=['_id', 'text'])
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 2 columns):
_id     56156 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 877.6+ KB


In [16]:
reviews.dropna(inplace=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56155 entries, 0 to 56155
Data columns (total 2 columns):
_id     56155 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 1.3+ MB


In [17]:
train_reviews = reviews['text'][0:2000]
train_reviews

0       Burger joint offers a wide range of cheeseburg...
1       It was really good. The mushroom broth was esp...
2       $8.90 for sesame rice, mushroom rendang, curry...
3       2 mains + 1 green rice bento from greendot. Re...
4       The lion mane mushroom rendang is so delicious...
                              ...                        
1995    Part of an all-you-can-eat vegan lunch set. Al...
1996    Part of an all-you-can-eat vegan lunch set. Th...
1997    Egg white omelette with asparagus, feta, sauté...
1998    This dish of stuffed tomatoes and green pepper...
1999    Yummy warm dessert with crumbled hazelnuts and...
Name: text, Length: 2000, dtype: object

In [0]:
ingredients_list = ['pistachio', 'almond', 'walnut', 'cashew', 'peanuts', 'macadamia', 'hazelnuts', 'pecan', 'brazil nut', 'pine nut' ]
issues_list = ['crumble - maybe a verb', 'milk']
food_times = 
to_be_added_list = ['spirulina', 'herbs', 'micro-greens', 'greens','spinach','eggs', 'nut butters', 'cream cheese', 'cake', 'pepper', 'zucchini', 'aubergines',  'tomato',\
                    'oysters', 'kale', 'cucumber', 'quinoa', 'tomatoes', 'onion' ,'garlic', 'avocado', 'chilli', 'dark chocolate', 'mushrooms', 'coriander' ,'corn chips',\
                    'marinara', 'donuts', 'olives', 'crackers', 'waffles', 'strawberry', 'blueberry', 'raspberry', 'compote', 'mayo', 'mayonnaise', 'soy milk', 'oat milk', \
                    'coconut milk', 'almond milk', 'caffeine', 'milk tea', 'cauliflower', 'matcha', 'sorbet', 'banana', 'goji berry', 'chickpea', 'spinach', 'romaine',\
                    'arugula', 'cacao', 'Portobello', 'pear', 'plant milk', 'BBQ sauce', 'artichoke', 'mango', 'sandwich', 'capsicum', 'wedge', 'kang kong', 'seitan',\
                    'ciabatta', 'focaccia', 'bean sprouts', 'olive oil']

In [0]:
check = np.random.randint(0,50000,3)
print(check)

for doc in list(nlp.pipe(texts[check])):
  displacy.render(doc,'ent',jupyter=True)
#  displacy.render(doc,'dep',jupyter=True,options={'distance': 60, 'fine_grained': True})



[49671 32259 46827]


In [0]:
doc = nlp("I like London. I also like Bangalore")
tok_list = [token for token in doc]
print(tok_list, len(tok_list))


[I, like, London, ., I, also, like, Bangalore] 8


In [0]:
tags = ['O' for tok in tok_list]
tags

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [0]:
tags[2] = 'U-LOC'
tags[7] = 'U-LOC'
tags

['O', 'O', 'U-LOC', 'O', 'O', 'O', 'O', 'U-LOC']

In [0]:
entities = offsets_from_biluo_tags(doc, tags)
entities

[(7, 13, 'LOC'), (27, 36, 'LOC')]

In [0]:
doc = nlp("I love Punjabi cuisine.")
[ent.label_ for ent in doc.ents]

['NORP']

['NORP']

In [0]:
tok_list = [token for token in doc]
print(tok_list, len(tok_list))
tags = ['O' for tok in tok_list]
print(tags)

[I, love, Punjabi, cuisine, .] 5
['O', 'O', 'O', 'O', 'O']


In [0]:
tags[2] = 'U-NORP'
print(tags)
entities = offsets_from_biluo_tags(doc, tags)
print(entities)


['O', 'O', 'U-NORP', 'O', 'O']
[(7, 15, 'NORP')]


['O', 'O', 'U-LOC', 'O']

In [0]:
json_data = docs_to_json([doc])
json_data['paragraphs'][0]['sentences'][0]['tokens'][2]['ner']

'U-NORP'

In [0]:
import srsly

In [0]:
f = open('rice.txt')
docs = [] # initialize a list to be populated wih strings
lines = f.readlines()  # returns a list of srings from the txt file
for line in lines:
    # print(line[:]) # display the sentence from that line
    doc = nlp(line) # convert string into a spacy doc object using nlp
    docs.append(doc) # add new doc to the list of docs

json_data = docs_to_json(docs) # convert doc into a json file
srsly.write_json('spacy_train.json', [json_data])


In [0]:
f.close()

In [0]:
check_list = [('bok','U_vegetable'),('choy','U_vegetable'),\
              ('bean','U_vegetable'), ('sprouts','U_vegetable'),\
              ('soffritto','U_dish'),('patties','U_dish'),('cakes','U_dish'),('parmesan','U_dish'),('cheese','U_dish'),('breadcrumbs','U_dish'),\
              ('soy','U_condiment'),('sauce','U_condiment'),\
              ('olive','U_condiment'),('oil','U_condiment'),\
              ('wine','U_beverage'),\
              ('3/4-inch','U_QUANTITY'), ('2-inch','U_QUANTITY'),('3-inch','U_QUANTITY'),('6-inch','U_QUANTITY'),\
              ('first','U_ORDINAL'),('second','U_ORDINAL'),('third','U_ORDINAL'),('fourth','U_ORDINAL'),('fifth','U_ORDINAL'),('last','U_ORDINAL'),\
              ('1','U_CARDINAL'),('2','U_CARDINAL'),('3','U_CARDINAL'),('4','U_CARDINAL'),('5','U_CARDINAL'),('6','U_CARDINAL'),('10','U_CARDINAL'),\
              ('100','U_CARDINAL'),('one','U_CARDINAL'),('two','U_CARDINAL'),('three','U_CARDINAL'),('four','U_CARDINAL'),('five','U_CARDINAL'),('six','U_CARDINAL'),\
              ('seven','U_CARDINAL'),('eight','U_CARDINAL'),('nine','U_CARDINAL'),('ten','U_CARDINAL'),\
              ('minute','U_TIME'),('minutes','U_TIME'),('seconds','U_TIME'),('hour','U_TIME'),('hours','U_TIME'),('day','U_TIME'),\
              ('$', 'U_MONEY'), ('USD', 'U_MONEY'),('SGD', 'U_MONEY'),('S$', 'U_MONEY'), ('Rs', 'U_MONEY'), ('£', 'U_MONEY'), ('€', 'U_MONEY')]

In [0]:
''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == 'rice'])))

'U_dish'

In [0]:
f = open('rice.txt')
docs = [] # initialize a list to be populated wih strings
lines = f.readlines()  # returns a list of srings from the txt file
for line in lines:
    # print(line[:]) # display the sentence from that line
    doc = nlp(line) # convert string into a spacy doc object using nlp
    docs.append(doc) # add new doc to the list of docs
f.close()



In [0]:
train_gen = (line for line in open('rice.txt'))
type(train_gen)
to_train_ents = []

In [56]:
l = next(train_gen)
tok_list = [token.text for token in nlp(l)] 
#print(l, tok_list)
tag_list = ['O' for token in nlp(l)]
#print(tag_list)
matched_ents = [(i,item) for i, item in enumerate(tok_list) if item.lower() in [i[0] for i in ent_list + check_list]] 
#print(matched_ents)

for ent in matched_ents:
  ent_tag = ''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == ent[1].lower()])))
  print(ent[0], ent[1], ent_tag)
  tag_list[ent[0]] = ent_tag

#print(tag_list)
entities = offsets_from_biluo_tags(nlp(l), tag_list)
print(entities)
to_train_ents.append((l, dict(entities = entities)))
displacy.render(nlp(l),'ent',jupyter=True)
print(l, entities)

#displacy.render(l,'dep',jupyter=True)

[]


Leave alone for five to seven minutes, then serve. []


In [57]:
len(to_train_ents)
#to_train_ents[-5:]

37

In [58]:
to_train_ents[0:5]

[('If using tofu, cook it.\n', {'entities': [(9, 13, 'dish')]}),
 ('Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.\n',
  {'entities': [(9, 13, 'dish'), (83, 87, 'dish'), (137, 148, 'dish')]}),
 ('If using onions, dice them into whatever size you prefer.\n',
  {'entities': [(9, 15, 'vegetable')]}),
 ('If using bok choy, separate the stalk part from the leaf part. Slice the leaf part crosswise into 3/4-inch-thick strips, then tear the strips into 2-inch pieces. These pieces will be treated much like the bean sprouts, though perhaps with slightly more cooking. The stalk part will be treated as celery.\n',
  {'entities': [(9, 17, 'vegetable'),
    (98, 106, 'QUANTITY'),
    (147, 153, 'QUANTITY'),
    (205, 217, 'vegetable'),
    (296, 302, 'vegetable')]}),
 ('If using bean sprouts, wash them and discard any that are not white and crunchy.\n',
  {'entitie

In [0]:
import itertools
merged = list(itertools.chain(*[tup[1]['entities'] for tup in to_train_ents]))

In [60]:
[t[2] for t in merged]

['dish',
 'dish',
 'dish',
 'dish',
 'vegetable',
 'vegetable',
 'QUANTITY',
 'QUANTITY',
 'vegetable',
 'vegetable',
 'vegetable',
 'dish',
 'condiment',
 'dish',
 'dish',
 'condiment',
 'dish',
 'condiment',
 'dish',
 'vegetable',
 'vegetable',
 'condiment',
 'CARDINAL',
 'condiment',
 'condiment',
 'dish',
 'vegetable',
 'vegetable',
 'dish',
 'dish',
 'vegetable',
 'condiment',
 'condiment',
 'dish',
 'dish',
 'condiment',
 'condiment',
 'beverage',
 'dish',
 'dish',
 'beverage',
 'dish',
 'dish',
 'dish',
 'dish',
 'dish',
 'dish',
 'CARDINAL',
 'dish',
 'condiment',
 'condiment',
 'condiment',
 'condiment',
 'dish',
 'dish',
 'dish',
 'dish',
 'CARDINAL',
 'dish',
 'dish',
 'dish',
 'condiment',
 'vegetable',
 'vegetable',
 'dish',
 'dish',
 'dish',
 'dish',
 'dish',
 'dish',
 'ORDINAL',
 'condiment',
 'herb',
 'dish',
 'dish',
 'spice',
 'dish',
 'vegetable',
 'vegetable',
 'vegetable',
 'dish',
 'dish',
 'dish',
 'dish',
 'dish',
 'ORDINAL',
 'TIME',
 'CARDINAL',
 'dish',
 'dis

In [61]:
model = 'en_core_web_lg'



"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

# add labels
for _, annotations in to_train_ents:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

Loaded model 'en_core_web_lg'


In [62]:
ner.labels

('dish',
 'ORG',
 'condiment',
 'FAC',
 'vegetable',
 'ORDINAL',
 'PERSON',
 'PRODUCT',
 'LOC',
 'spice',
 'CARDINAL',
 'LANGUAGE',
 'TIME',
 'WORK_OF_ART',
 'GPE',
 'DATE',
 'LAW',
 'NORP',
 'beverage',
 'MONEY',
 'EVENT',
 'herb',
 'PERCENT',
 'QUANTITY')

In [0]:
from spacy.gold import GoldParse

In [66]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ["ner", "entity_ruler","merge_entities"]]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()



    for itn in range(5):
      random.shuffle(clean_train_ents)
      losses = {}

#      for item in clean_train_ents:
# batch up the examples using spaCy's minibatch
      batches = minibatch(clean_train_ents, size=compounding(4.0, 32.0, 1.001))

      for batch in batches:
        texts, annotations = zip(*batch)
#        print([item[0]], [item[1]])

#        print(texts)
#        print(annotations)
        nlp.update(texts, annotations, drop=0.5, losses=losses)
        print(losses)

{'ner': 68.43624067306519}
{'ner': 184.61740112304688}
{'ner': 258.61230278015137}
{'ner': 335.3837299346924}
{'ner': 397.74806785583496}
{'ner': 496.07928681373596}
{'ner': 604.3904078006744}
{'ner': 697.9849517345428}
{'ner': 898.3083918094635}
{'ner': 913.0961752086878}
{'ner': 77.2951397895813}
{'ner': 199.11220693588257}
{'ner': 272.8479437828064}
{'ner': 399.1770589351654}
{'ner': 510.91890597343445}
{'ner': 571.7339107990265}
{'ner': 673.7506942749023}
{'ner': 836.8600540161133}
{'ner': 893.0359942913055}
{'ner': 904.4475624634651}
{'ner': 76.6790599822998}
{'ner': 248.98345279693604}
{'ner': 359.4646096229553}
{'ner': 447.50255060195923}
{'ner': 550.7681419849396}
{'ner': 665.9656307697296}
{'ner': 718.2194759845734}
{'ner': 780.4857132434845}
{'ner': 906.0226509571075}
{'ner': 942.5896518924274}
{'ner': 68.84220838546753}
{'ner': 148.96541571617126}
{'ner': 251.8399977684021}
{'ner': 350.8443260192871}
{'ner': 446.8515214920044}
{'ner': 509.65203619003296}
{'ner': 614.89673566

In [0]:
check_list = []
to_train_ents = []

In [23]:
train_rev = (rev for rev in train_reviews)
type(train_rev)

generator

In [171]:
rev = next(train_rev)
tok_list = [token.text for token in nlp(rev)] 
#print(rev, tok_list)
tag_list = ['O' for token in nlp(rev)]
#print(tag_list)
matched_ents = [(i,item) for i, item in enumerate(tok_list) if item.lower() in [i[0] for i in ent_list + check_list]] 
#print(matched_ents)

ent_for_doc = []
for ent in matched_ents:
  ent_tag = ''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == ent[1].lower()])))
  ent_for_doc.append((ent[0], ent[1], ent_tag))  
  tag_list[ent[0]] = ent_tag
print(ent_for_doc)
#print(tag_list)
entities = offsets_from_biluo_tags(nlp(rev), tag_list)
print(entities)
to_train_ents.append((rev, dict(entities = entities)))
displacy.render(nlp(rev),'ent',jupyter=True)
#print(rev, entities)

#displacy.render(l,'dep',jupyter=True)

n_entities_to_be_added = int(input("How many entities are missing: "))
if n_entities_to_be_added == 0:
  print('You entered {} Thanks. So no new entities to be considered'.format(n_entities_to_be_added))
else:
  for user_input in range(n_entities_to_be_added):
    e_name = input('Provide the name of the entity to be added: ')
    e_label = input('Provide the label of the entity to be added:')
    print(e_name, e_label)
    check_list.append((e_name, e_label))

[(2, 'salad', 'U_dish'), (5, 'Autobus', 'ORG')]
[(13, 18, 'dish')]


How many entities are missing: 0
You entered 0 Thanks. So no new entities to be considered


In [173]:
check_list[-5:]

[('brussels sprouts', 'U_vegetable'),
 ('soy sesame', 'U_condiment'),
 ('soya sesame', 'U_condiment'),
 ('squash', 'U_vegetable'),
 ('honey mustard sauce', 'U_condiment')]

In [172]:
len(to_train_ents)

109

In [175]:
model = 'en_core_web_lg'



"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

# add labels
for _, annotations in to_train_ents:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

Loaded model 'en_core_web_lg'


In [176]:
ner.labels

('PERCENT',
 'CARDINAL',
 'spice',
 'pastryU_dish',
 'vegetable',
 'PERSON',
 'herb',
 'bread',
 'dish',
 'herbU_vegetable',
 'PRODUCT',
 'GPE',
 'fruitU_vegetable',
 'EVENT',
 'WORK_OF_ART',
 'ORDINAL',
 'ORG',
 'MONEY',
 'FAC',
 'LANGUAGE',
 'LAW',
 'QUANTITY',
 'condiment',
 'fruit',
 'NORP',
 'breadU_dish',
 'TIME',
 'DATE',
 'LOC',
 'spiceU_vegetable')

In [99]:
len(to_train_ents)

63

In [177]:
clean_train_ents = trim_entity_spans(to_train_ents)

Burger joint offers a wide range of cheeseburger choices and vegetarian mushroom option. Price range from $12 for the standard cheeseburger to the top of the range double jalapeno bacon cheeseburger at $30.40.

We ordered the vegetarian option which is the Portobello mushroom burger with the option to add jalapeno. Inside the bun it comes with tomato, onions, pickles and lettuce, basically what you expect from a burger. Currently burger joint is only offering the Portobello mushroom burger as their only choice of vegetarian option but I spoke to Darius, the manager who said that they will be sampling more vegetarian options in the coming weeks. I asked why they are bringing in more vegetarian options is because there is a increase demand for vegetarian dishes. 

Waiting time for the he Portobello burger was about 10 minutes and it came In a paper bag and it was well wrapped inside. When you unwrap the burger, you can just imagine how huge the burger is and it can barely fit into your m

In [190]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ["ner", "entity_ruler","merge_entities"]]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()



    for itn in range(100):
      random.shuffle(clean_train_ents)
      losses = {}

#      for item in clean_train_ents:
# batch up the examples using spaCy's minibatch
      batches = minibatch(clean_train_ents, size=compounding(10.0, 32.0, 1.001))

      for batch in batches:
        texts, annotations = zip(*batch)
#        print([item[0]], [item[1]])

#        print(texts)
#        print(annotations)
        nlp.update(texts, annotations, drop=0.5, losses=losses)
        print(losses)

{'ner': 359.6167507171631}
{'ner': 1044.9250736236572}
{'ner': 1594.313588142395}
{'ner': 1989.6091718673706}
{'ner': 2625.0646419525146}
{'ner': 3091.6135540008545}
{'ner': 3961.095869064331}
{'ner': 4274.681268692017}
{'ner': 4704.154732704163}
{'ner': 5051.164274215698}
{'ner': 5524.894720077515}
{'ner': 465.1182155609131}
{'ner': 988.834508895874}
{'ner': 1494.3885164260864}
{'ner': 2069.1512594223022}
{'ner': 2675.306851387024}
{'ner': 3158.2569398880005}
{'ner': 3542.4446363449097}
{'ner': 4173.406796455383}
{'ner': 4478.461621284485}
{'ner': 5041.80259513855}
{'ner': 5582.517515182495}
{'ner': 441.93192863464355}
{'ner': 896.1393356323242}
{'ner': 1584.5796241760254}
{'ner': 1975.0675654411316}
{'ner': 2267.5771946907043}
{'ner': 2858.408874988556}
{'ner': 3776.0630745887756}
{'ner': 4237.513055324554}
{'ner': 4621.016045093536}
{'ner': 5037.430918216705}
{'ner': 5475.250187397003}
{'ner': 255.0232982635498}
{'ner': 559.0359325408936}
{'ner': 1127.0450916290283}
{'ner': 1733.458

In [191]:
nlp.pipe_names


['tagger', 'parser', 'entity_ruler', 'ner', 'merge_entities']

In [184]:
ruler = EntityRuler(nlp).from_disk("patterns.jsonl")
nlp.add_pipe(ruler, before='ner')
merge_ents = nlp.create_pipe("merge_entities")
nlp.add_pipe(merge_ents, after='ner')
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'merge_entities']

In [192]:
test_docs = train_reviews[100:120]

for doc in test_docs:
  displacy.render(nlp(doc),'ent',jupyter=True)

In [0]:
pwd

'/gdrive/My Drive/abv_reviews'

In [108]:
check_list

[('bok', 'U_vegetable'),
 ('choy', 'U_vegetable'),
 ('bean', 'U_vegetable'),
 ('sprouts', 'U_vegetable'),
 ('soffritto', 'U_dish'),
 ('patties', 'U_dish'),
 ('cakes', 'U_dish'),
 ('parmesan', 'U_dish'),
 ('cheese', 'U_dish'),
 ('breadcrumbs', 'U_dish'),
 ('soy', 'U_condiment'),
 ('sauce', 'U_condiment'),
 ('olive', 'U_condiment'),
 ('oil', 'U_condiment'),
 ('wine', 'U_beverage'),
 ('3/4-inch', 'U_QUANTITY'),
 ('2-inch', 'U_QUANTITY'),
 ('3-inch', 'U_QUANTITY'),
 ('6-inch', 'U_QUANTITY'),
 ('first', 'U_ORDINAL'),
 ('second', 'U_ORDINAL'),
 ('third', 'U_ORDINAL'),
 ('fourth', 'U_ORDINAL'),
 ('fifth', 'U_ORDINAL'),
 ('last', 'U_ORDINAL'),
 ('1', 'U_CARDINAL'),
 ('2', 'U_CARDINAL'),
 ('3', 'U_CARDINAL'),
 ('4', 'U_CARDINAL'),
 ('5', 'U_CARDINAL'),
 ('6', 'U_CARDINAL'),
 ('10', 'U_CARDINAL'),
 ('100', 'U_CARDINAL'),
 ('one', 'U_CARDINAL'),
 ('two', 'U_CARDINAL'),
 ('three', 'U_CARDINAL'),
 ('four', 'U_CARDINAL'),
 ('five', 'U_CARDINAL'),
 ('six', 'U_CARDINAL'),
 ('seven', 'U_CARDINAL'),
 ('

In [0]:
nlp.to_disk('trained_model_2020_01_06')

In [0]:
import json

with open('checklist.txt', 'w') as f:
    f.write(json.dumps(check_list))

In [114]:
[token.pos_ for token in nlp("lion's mane mushroom falafels")]

['NOUN', 'PART', 'NOUN', 'NOUN', 'NOUN']

In [0]:
for doc in list(nlp.pipe(texts[check])):
  displacy.render(doc,'ent',jupyter=True)
#  displacy.render(doc,'dep',jupyter=True,options={'distance': 60, 'fine_grained': True})